In [1]:
from numpy.random import default_rng
from scapy.all import Ether, Dot1Q, RawPcapWriter, IP, UDP

/usr/lib/python3.10/site-packages/scapy/layers/ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/usr/lib/python3.10/site-packages/scapy/layers/ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


In [2]:
# create new Generator:
rng = default_rng()

In [3]:
def get_header(payload, pkt_id, vlan_prio, eth_dst, eth_src):
    return Ether(dst=eth_dst,
                 src=eth_src
                 )/Dot1Q(prio=vlan_prio,
                         vlan=VLAN_ID,
                         type=VLAN_TYPE
                         )/IP(id=pkt_id,
                              version=4,
                              dst='10.0.0.1',
                              src='10.0.0.2')/UDP(sport=10002,
                                                  dport=10001)/payload


def inc_pkt_id(pkt_id):
    if pkt_id >= 65535:
        return 0
    else:
        return pkt_id + 1

In [4]:
# onboard i211:
MAC_NODE0 = "1C:1B:0D:9A:70:8C"

# dedicated i210:
MAC_NODE1 = "00:00:00:00:00:01"
MAC_NODE2 = "00:00:00:00:00:02"
MAC_NODE3 = "00:00:00:00:00:03"

VLAN_ID = 3
VLAN_TYPE = 0x0800
HEADERS = 18 + 20 + 8 + 4  # Eth, VLAN, IP, UDP, CRC
FULL_FRAME = 1522
MIN_FRAME = 68

DURATION = 60

In [5]:
def create_pcap(length, cycle, vlan_prio, eth_src, eth_dst, validation_test_no):
    path = "/home/arch/testbed_src/src/pcap_writer/cut-through.pcap"
    pcap = RawPcapWriter(open(path, 'wb'), nano=False, linktype=1, sync=True, append=True)

    sec = 0
    usec = 0
    pkt_id = 0

    while sec < length:
        packet_size = round(rng.uniform(64, 1522))
        p = (packet_size - HEADERS) * '0'
        header = get_header(payload=p,
                            pkt_id=pkt_id,
                            vlan_prio=vlan_prio,
                            eth_dst=eth_dst,
                            eth_src=eth_src)
        pkt_id = inc_pkt_id(pkt_id)

        pcap.write_header(header)
        pcap.write_packet(header, sec=sec, usec=usec)

        usec += cycle
        if usec >= 1000000:
            sec += 1
            usec -= 1000000

In [6]:
create_pcap(length=DURATION,
            cycle=100,  # 100µs between frames
            vlan_prio=6,
            eth_src=MAC_NODE1,
            eth_dst=MAC_NODE0,
            validation_test_no=4
            )

# Measurement:
- copy file to node1/2
- start capturing on node0 with tcpdump:
`sudo tcpdump -i enp5s0 ether src 68:05:ca:71:d0:94 and udp port 10001 -j adapter_unsynced --time-stamp-precision=nano -s 250 -w validation-recv-4.pcap`
- start sending on node1/2 with tcpreplay:
`sudo tcpreplay -i enp1s0 --timer=nano -K --sockprio=2 --loop=1 validation-src-4.pcap`
- copy files back to hp-workstation for evaluating the measurement